In [10]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
import numpy as np, random, os

In [5]:
dataset1 = 'datasets/FashionMNIST0.3.npz'
dataset2 = 'datasets/FashionMNIST0.6.npz'
dataset3 = 'datasets/CIFAR.npz'

data = np.load(dataset2)

print(data.keys())

Xtr = data['Xtr']
Str = data['Str']
Xts = data['Xts']
Yts = data['Yts']

print("Training data shape:", Xtr.shape)
print("Training labels shape:", Str.shape)
print("Test data shape:", Xts.shape)
print("Test labels shape:", Yts.shape)

KeysView(NpzFile 'datasets/FashionMNIST0.6.npz' with keys: Xtr, Str, Xts, Yts)
Training data shape: (18000, 784)
Training labels shape: (18000,)
Test data shape: (3000, 784)
Test labels shape: (3000,)


In [ ]:
def estimate_T_by_repeats(X, s, n_classes=3, R=10, seed=0):
    rng = np.random.RandomState(seed)
    T_counts = np.zeros((n_classes, n_classes), dtype=float)
    for r in range(R):
        X_tr, X_va, s_tr, s_va = train_test_split(
            X, s, test_size=0.2, stratify=s, random_state=seed + r
        )
        clf = RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=seed + r)
        y_hat = clf.fit(X_tr, s_tr).predict(X_va)

        for i in range(n_classes):
            mask = (y_hat == i)
            if mask.sum() == 0:
                T_counts[i] += 1.0   # 平滑时等价于 +[1,1,1]
            else:
                T_counts[i] += np.bincount(s_va[mask], minlength=n_classes)

    # 轻度平滑，避免 0：可改成 +1 更强
    T_counts += 1e-6
    T_hat = T_counts / T_counts.sum(axis=1, keepdims=True)
    return T_hat

In [ ]:
data = np.load(dataset1)
X = data['Xtr'].reshape(len(data['Xtr']), -1)
s = data['Str']

seed = 10086
np.random.seed(seed); random.seed(seed); os.environ["PYTHONHASHSEED"]=str(seed)

T_hat = estimate_T_by_repeats(X, s, n_classes=3, R=10, seed=seed)
print(np.round(T_hat, 2))

[[0.8 0.2 0. ]
 [0.  0.8 0.2]
 [0.2 0.  0.8]]


In [17]:
data = np.load(dataset2)
X_tr1 = data['Xtr']
S_tr1 = data['Str']

X_tr1_flatten = X_tr1.reshape((X_tr1.shape[0], -1))

X_train, X_val, S_train, S_val = train_test_split(X_tr1_flatten, S_tr1, test_size=0.2, random_state=42)

n_classes = 3
T = np.zeros((n_classes, n_classes))

seed = 10086
np.random.seed(seed); random.seed(seed); os.environ["PYTHONHASHSEED"] = str(seed)

#model = LogisticRegression(max_iter=500) 0.6
#model = RandomForestClassifier() 0.7   
#model = make_pipeline(StandardScaler(),SVC()) 0.7
model = RandomForestClassifier(n_estimators=500, random_state=seed, n_jobs=-1)
X_train, X_val, S_train, S_val = train_test_split(
    X_tr1_flatten, S_tr1, test_size=0.2, stratify=S_tr1, random_state=seed
)

pre_labels = model.fit(X_train, S_train).predict_proba(X_val)

for i in range(3):
    mask = (pre_labels == i)
    cnt = np.bincount(S_val[mask], minlength=3).astype(float)
    T[i] = cnt / cnt.sum()
    

T = T / T.sum(axis=1, keepdims=True)

print(np.round(T, 1)) 

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [14]:
data = np.load(dataset3)
X_tr1 = data['Xtr']
S_tr1 = data['Str']

X_tr1_flatten = X_tr1.reshape((X_tr1.shape[0], -1))

X_train, X_val, S_train, S_val = train_test_split(X_tr1_flatten, S_tr1, test_size=0.2, random_state=42)

n_classes = 3
T = np.zeros((n_classes, n_classes))

seed = 10086
np.random.seed(seed); random.seed(seed); os.environ["PYTHONHASHSEED"] = str(seed)

#model = LogisticRegression(max_iter=500) 0.6
#model = RandomForestClassifier() 0.7   
#model = make_pipeline(StandardScaler(),SVC()) 0.7
model = ExtraTreesClassifier(n_estimators=500, random_state=seed, n_jobs=-1)
X_train, X_val, S_train, S_val = train_test_split(
    X_tr1_flatten, S_tr1, test_size=0.2, stratify=S_tr1, random_state=seed
)

pre_labels = model.fit(X_train, S_train).predict_proba(X_val)

for i in range(3):
    idx = np.argsort(pre_labels[:, i])[-max(1, int(0.03*len(S_val))):]
    T[i] = np.mean(pre_labels[idx], axis=0)

T = T / T.sum(axis=1, keepdims=True)

print(np.round(T, 1)) 

[[0.5 0.3 0.2]
 [0.3 0.5 0.2]
 [0.3 0.3 0.5]]


In [8]:
data2 = np.load(dataset2)
X_tr2 = data2['Xtr']
S_tr2 = data2['Str']

X_tr2_flatten = X_tr2.reshape((X_tr2.shape[0], -1))

X_train, X_val, S_train, S_val = train_test_split(X_tr2_flatten, S_tr2, test_size=0.2, random_state=42)

n_classes = 3
T2 = np.zeros((n_classes, n_classes))

#model = LogisticRegression(max_iter=500) 0.6
#model = RandomForestClassifier() 0.7
#model = make_pipeline(StandardScaler(),SVC()) 0.7
model = ExtraTreesClassifier()
model.fit(X_train, S_train)

pre_labels = model.predict(X_val)

for i in range(len(S_val)):
    noi_label = S_val[i]
    pre_label = pre_labels[i]
    T2[pre_label, noi_label] += 1

T2 = T2 / T2.sum(axis=1, keepdims=True)

T2_orig = T2.copy()
T2 = np.round(T2, 1)

row_sums = T2.sum(axis=1)
rows_to_fix = np.where(np.isclose(row_sums, 0.9))[0]

for r in rows_to_fix:
    c = int(np.argmax(T2_orig[r]))
    T2[r, c] = np.round(T2[r, c] + 0.1, 1)

print(T2)

[[0.3 0.4 0.3]
 [0.3 0.4 0.3]
 [0.3 0.3 0.4]]


In [7]:
data3 = np.load(dataset3)
X_tr3 = data3['Xtr']
S_tr3 = data3['Str']

X_tr3_flatten = X_tr3.reshape((X_tr3.shape[0], -1))

X_train, X_val, S_train, S_val = train_test_split(X_tr3_flatten, S_tr3, test_size=0.2, random_state=42)

n_classes = 3
T3 = np.zeros((n_classes, n_classes))

#model = LogisticRegression(max_iter=500) 0.6
#model = RandomForestClassifier() 0.7
#model = make_pipeline(StandardScaler(),SVC()) 0.7
model = ExtraTreesClassifier()
model.fit(X_train, S_train)

pre_labels = model.predict(X_val)

for i in range(len(S_val)):
    noi_label = S_val[i]
    pre_label = pre_labels[i]
    T3[pre_label, noi_label] += 1

T3 = T3 / T3.sum(axis=1, keepdims=True)

T3_orig = T3.copy()
T3 = np.round(T3, 1)

row_sums = T3.sum(axis=1)
rows_to_fix = np.where(np.isclose(row_sums, 0.9))[0]

for r in rows_to_fix:
    c = int(np.argmax(T3_orig[r]))
    T3[r, c] = np.round(T3[r, c] + 0.1, 1)

print(T3)

[[0.3 0.4 0.3]
 [0.3 0.4 0.3]
 [0.3 0.3 0.4]]
